### RAG

In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.ui import Console
from autogen_agentchat.messages import AgentEvent, ChatMessage, TextMessage
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_core import CancellationToken
from autogen_core.memory import ListMemory, MemoryContent, MemoryMimeType
from autogen_ext.memory.chromadb import ChromaDBVectorMemory, PersistentChromaDBVectorMemoryConfig
import os

from dotenv import load_dotenv
from indexers import SimpleDocumentIndexer

In [2]:
load_dotenv(override=True)

azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

if not all([azure_openai_api_key, azure_openai_endpoint, azure_openai_deployment_name, azure_openai_api_version]):
    raise ValueError("""
        Please set all required environment variables: 
            AZURE_OPENAI_API_KEY, 
            AZURE_OPENAI_ENDPOINT, 
            AZURE_OPENAI_DEPLOYMENT_NAME, 
            AZURE_OPENAI_API_VERSION
        """)


In [3]:
model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_openai_deployment_name,
    model=azure_openai_deployment_name,
    api_key=azure_openai_api_key,
    azure_endpoint=azure_openai_endpoint,
    api_version=azure_openai_api_version,
)

In [4]:
chroma_user_memory = ChromaDBVectorMemory(
    config=PersistentChromaDBVectorMemoryConfig(
        collection_name="autogen_documentation",
        persistent_path="./autgen_chroma_db",
        k=3,
        score_threshold=0.4,
    )
)

await chroma_user_memory.clear()

In [5]:
async def index_autogen_docs():
    indexer = SimpleDocumentIndexer(
        memory=chroma_user_memory,
    )
    
    documents=[
            "https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/agents.html",
            "https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/teams.html",
            "https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/termination.html",
            "https://raw.githubusercontent.com/microsoft/autogen/refs/heads/main/README.md"
    ]
    
    chunks: int = await indexer.index_documents(documents)
    
    print(f"Indexed {chunks} chunks from {len(documents)} documents.")

In [6]:
await index_autogen_docs()

Indexed 69 chunks from 4 documents.


In [8]:
rag_assistant = AssistantAgent(
    name="RAGAssistant",
    model_client=model_client,
    memory=[chroma_user_memory],
    system_message="You are a helpful assistant that can answer questions based on the provided context from the documentation.",
)

In [9]:
stream = rag_assistant.run_stream(
    task = "What is AutoGen AgentChat?",
)

await Console(stream)

---------- TextMessage (user) ----------
What is AutoGen AgentChat?
---------- MemoryQueryEvent (RAGAssistant) ----------
[MemoryContent(content='arch Literature Review API Reference PyPi Source AgentChat Agents Agents # AutoGen AgentChat provides a set of preset Agents, each with variations in how an agent might respond to messages. All agents share the following attributes and methods: name : The unique name of the agent. description : The description of the agent in text. run : The method that runs the agent given a task as a string or a list of messages, and returns a TaskResult . Agents are expected to be stateful and this method is expected to be called with new messages, not complete history . run_stream : Same as run() but returns an iterator of messages that subclass BaseAgentEvent or BaseChatMessage followed by a TaskResult as the last item. See autogen_agentchat.messages for more information on AgentChat message types. Assistant Agent # AssistantAgent is a built-in agent tha

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 13, 21, 59, 43, 293242, tzinfo=datetime.timezone.utc), content='What is AutoGen AgentChat?', type='TextMessage'), MemoryQueryEvent(source='RAGAssistant', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 13, 21, 59, 43, 736479, tzinfo=datetime.timezone.utc), content=[MemoryContent(content='arch Literature Review API Reference PyPi Source AgentChat Agents Agents # AutoGen AgentChat provides a set of preset Agents, each with variations in how an agent might respond to messages. All agents share the following attributes and methods: name : The unique name of the agent. description : The description of the agent in text. run : The method that runs the agent given a task as a string or a list of messages, and returns a TaskResult . Agents are expected to be stateful and this method is expected to be called with new messages, not complete history . run_str